In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

C:\Users\Colin\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Colin\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Colin\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Colin\Anaconda3\envs\PythonData\lib\site-packages

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


* What variable(s) are considered the target for your model?
    - The target for my model is the 'IS_SUCCESSFUL' variable.
    - This variable is defined by Alphabet Soup as whether the money was used effectively.

* What variable(s) are considered to be the features for your model?
    - Some variables I will be considering for features for my model are 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', and 'ASK_AMT'.
    
* What variable(s) are neither and should be removed from the input data?
    - I don't expect 'EIN' or 'NAME' to provide any explanatory power, and will remove them from the input data.

In [2]:
# Drop the unwanted variables from the data
charity_input = charity_df.drop(columns = ['EIN', 'NAME'])
charity_input.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Generate our categorical variable list
charity_cat = charity_input.dtypes[charity_input.dtypes == "object"].index.tolist()
charity_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [4]:
# Check the number of unique values in each column
charity_input[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [5]:
# Define a function that will redefine all values in a given column below a given count as "Other"
def bucketing(df, column = "APPLICATION_TYPE", cutoff = 1000, bucket_name = "Other"):
    # Get the value counts of given column
    counts = df[column].value_counts()
    
    # Determine which values to replace
    replace_values = list(counts[counts < cutoff].index)
    
    # Replace in DataFrame
    for value in replace_values:
        df[column] = df[column].replace(value, bucket_name)
        
    return df

In [6]:
# Use the function to redefine 'APPLICATION_TYPE' categories below 1000 as "Other"
charity_bucketed = bucketing(charity_input, "APPLICATION_TYPE", 1000)

# Check to make sure binning was successful
charity_bucketed['APPLICATION_TYPE'].value_counts()

# Cutting off below 1000 leaves 6 categories, which seems reasonable.

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [7]:
charity_bucketed = bucketing(charity_bucketed, 'CLASSIFICATION', 500)

# Check to make sure binning was successful
charity_bucketed['CLASSIFICATION'].value_counts()

# Cutting off below 500 leaves 7 categories, which seems reasonable.

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [8]:
charity_bucketed = bucketing(charity_bucketed, 'AFFILIATION', 100)

# Check to make sure binning was successful
charity_bucketed['AFFILIATION'].value_counts()

# Cutting off below 100 leaves 3 categories, including an "Other" with 114 values.
# This seems low, especially since the next smallest category has over 15000.
# I may combine them later for only two categories, but will leave as three for now.

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [9]:
charity_bucketed = bucketing(charity_bucketed, 'USE_CASE', 500)

# Check to make sure binning was successful
charity_bucketed['USE_CASE'].value_counts()

# Cutting off below 500 leaves 3 categories, which seems reasonable.

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [10]:
charity_bucketed = bucketing(charity_bucketed, 'ORGANIZATION', 500)

# Check to make sure binning was successful
charity_bucketed['ORGANIZATION'].value_counts()

# Cutting off below 500 leaves 3 categories, which seems reasonable.

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [11]:
# Since 'INCOME_AMT' might have some ordinal information, I want to do more than simply have an "Other" bucket.
column = 'INCOME_AMT'

# Get the value counts of given column
#counts = charity_bucketed[column].value_counts()

# Determine which values to replace
replace_values = ["10M-50M", "5M-10M", "50M+"]

# Replace in DataFrame
for value in replace_values:
    charity_bucketed[column] = charity_bucketed[column].replace(value, "5M+")

# Check to make sure binning was successful
charity_bucketed['INCOME_AMT'].value_counts()

# Each category having >500 values seems reasonable.

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_bucketed[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,CLASSIFICATION_C1000,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
charity_merged = charity_bucketed.merge(encode_df,left_index=True, right_index=True)
charity_merged = charity_merged.drop(charity_cat,1)
charity_merged.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y = charity_merged["IS_SUCCESSFUL"].values
X = charity_merged.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#X_train_scaled.head()

In [78]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 6
activ_func = "tanh"

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activ_func)
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activ_func))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 10)                340       
_________________________________________________________________
dense_29 (Dense)             (None, 6)                 66        
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 7         
Total params: 413
Trainable params: 413
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/cp.{acc:.4f}-{epoch:02d}.hdf5"

In [81]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='accuracy',
    verbose=1,
    #save_best_only=True,
    save_weights_only=False,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
24832/25724 [===========================>..] - ETA: 0s - loss: 0.5492 - acc: 0.7331
Epoch 00001: saving model to checkpoints/cp.0.7330-01.hdf5
25724/25724 [==============================] - 2s 91us/sample - loss: 0.5490 - acc: 0.7330
Epoch 2/100
25248/25724 [============================>.] - ETA: 0s - loss: 0.5484 - acc: 0.7329
Epoch 00002: saving model to checkpoints/cp.0.7329-02.hdf5
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5486 - acc: 0.7329
Epoch 3/100
24928/25724 [============================>.] - ETA: 0s - loss: 0.5482 - acc: 0.7323
Epoch 00003: saving model to checkpoints/cp.0.7322-03.hdf5
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5484 - acc: 0.7322
Epoch 4/100
24960/25724 [============================>.] - ETA: 0s - loss: 0.5486 - acc: 0.7324
Epoch 00004: saving model to checkpoints/cp.0.7325-04.hdf5
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5485 - acc: 0.7325
Epoch 5/100
2524

Epoch 34/100
25216/25724 [============================>.] - ETA: 0s - loss: 0.5447 - acc: 0.7351
Epoch 00034: saving model to checkpoints/cp.0.7350-34.hdf5
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5453 - acc: 0.7350
Epoch 35/100
25408/25724 [============================>.] - ETA: 0s - loss: 0.5458 - acc: 0.7347
Epoch 00035: saving model to checkpoints/cp.0.7347-35.hdf5
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5456 - acc: 0.7347
Epoch 36/100
25376/25724 [============================>.] - ETA: 0s - loss: 0.5454 - acc: 0.7356
Epoch 00036: saving model to checkpoints/cp.0.7355-36.hdf5
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5454 - acc: 0.7355
Epoch 37/100
25408/25724 [============================>.] - ETA: 0s - loss: 0.5458 - acc: 0.7343
Epoch 00037: saving model to checkpoints/cp.0.7346-37.hdf5
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5455 - acc: 0.7346
Epoch 38/100

25568/25724 [============================>.] - ETA: 0s - loss: 0.5441 - acc: 0.7360
Epoch 00067: saving model to checkpoints/cp.0.7358-67.hdf5
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5443 - acc: 0.7358
Epoch 68/100
25152/25724 [============================>.] - ETA: 0s - loss: 0.5445 - acc: 0.7360
Epoch 00068: saving model to checkpoints/cp.0.7361-68.hdf5
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5443 - acc: 0.7361
Epoch 69/100
25632/25724 [============================>.] - ETA: 0s - loss: 0.5445 - acc: 0.7350
Epoch 00069: saving model to checkpoints/cp.0.7352-69.hdf5
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5442 - acc: 0.7352
Epoch 70/100
24608/25724 [===========================>..] - ETA: 0s - loss: 0.5446 - acc: 0.7358
Epoch 00070: saving model to checkpoints/cp.0.7359-70.hdf5
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5443 - acc: 0.7359
Epoch 71/100
25696/25724 

25152/25724 [============================>.] - ETA: 0s - loss: 0.5426 - acc: 0.7375
Epoch 00100: saving model to checkpoints/cp.0.7364-100.hdf5
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5435 - acc: 0.7364
8575/8575 - 0s - loss: 0.5564 - acc: 0.7252
Loss: 0.5564416005103998, Accuracy: 0.7252478003501892


In [82]:
# Export our model to HDF5 file
nn.save("trained_model.h5")

In [22]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5552 - acc: 0.7283
Loss: 0.55519943630035, Accuracy: 0.7282798886299133
